In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Lemmatize edilmiş CSV dosyasını yükle
df = pd.read_csv('./data_without_extra_features/preprocessed_SEFACED.csv')

# Veriyi özellikler (lemmatize edilmiş metin) ve hedef değişken olarak ayır
df['cleaned_text'] = df['cleaned_text'].astype(str)

X = df['cleaned_text']
y = df['Class_Label']

# Etiketleri sayısal değerlere dönüştür
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Metinleri tokenle
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Maksimum dizi uzunluğunu tanımla
maxlen = 100

# Dizileri aynı uzunlukta olacak şekilde doldur
X_pad = pad_sequences(X_seq, maxlen=maxlen)

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# LSTM modelini tanımla
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Modeli derle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğit
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Modeli değerlendir

y_pred_prob = model.predict(X_test)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)

print(classification_report(y_test, y_pred))


Epoch 1/10
441/441 [==============================] - 200s 407ms/step - loss: 0.1339 - accuracy: 0.9431 - val_loss: 0.0260 - val_accuracy: 0.9921
Epoch 2/10
441/441 [==============================] - 172s 390ms/step - loss: 0.0114 - accuracy: 0.9965 - val_loss: 0.0201 - val_accuracy: 0.9932
Epoch 3/10
441/441 [==============================] - 174s 395ms/step - loss: 0.0039 - accuracy: 0.9991 - val_loss: 0.0235 - val_accuracy: 0.9915
Epoch 4/10
441/441 [==============================] - 173s 392ms/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 0.0277 - val_accuracy: 0.9904
Epoch 5/10
441/441 [==============================] - 104s 236ms/step - loss: 0.0029 - accuracy: 0.9992 - val_loss: 0.0300 - val_accuracy: 0.9921
Epoch 6/10
441/441 [==============================] - 96s 219ms/step - loss: 0.0029 - accuracy: 0.9992 - val_loss: 0.0354 - val_accuracy: 0.9901
Epoch 7/10
441/441 [==============================] - 99s 223ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.0347 - v